In [1]:
from collections import namedtuple
import torch
import torch.nn as nn
from torchvision import models
from utils.load_mvsa import *
from PIL import Image

from torchvision.transforms import ToTensor

In [ ]:
glove_data = load_mvsa_glove_data()
glove_data["train"][:5]

In [22]:
def load_image(_id):
    path = DIR + "images/" + _id + ".jpg"
    return Image.open(path)
im = load_image("11401")

In [42]:
class VGG16Extractor:
    def __init__(self, use_cuda=True):
        model = models.vgg16(pretrained=True)
        model.classifier = nn.Sequential(model.classifier[:-1])
        model.eval()
        if use_cuda:
            model = model.cuda()
        self.vgg = model
        self.use_cuda = use_cuda
        self.TARGET_SIZE = 224
        self.img2tensor = ToTensor()
    def extract(self, img):
        img = img.resize((self.TARGET_SIZE, self.TARGET_SIZE))
        X = self.img2tensor(img)
        X = X.unsqueeze(0)
        if self.use_cuda:
            X = X.cuda()
        feature = self.vgg(X).squeeze()
        feature = feature.cpu().detach().numpy()
        return feature
extractor = VGG16Extractor()

In [43]:
feature = extractor.extract(im)
feature.shape

(4096,)

In [47]:
def save_feature(_id, feature):
    path = DIR + "vgg16/" + _id + ".npy"
    np.save(path, feature)
# save_feature("11401", feature)

In [49]:
%%time
def extract_vgg16():
    for key in ["train", "valid", "test"]:
        for i in glove_data[key]:
            im = load_image(i["id"])
            feature = extractor.extract(im)
            save_feature(i["id"], feature)
extract_vgg16()

CPU times: user 19min 25s, sys: 13.7 s, total: 19min 39s
Wall time: 5min 5s


In [4]:
from utils.load_mvsa import *
import numpy as np

In [3]:
glove_data = load_mvsa_glove_data()

In [7]:
a = load_vgg(glove_data["train"][0]["id"])
a.shape

(4096,)

In [24]:
MEAN = np.zeros(4096, dtype=np.float32)
total = 0
for key in ["train", "valid"]:
    for i in glove_data[key]:
        MEAN += load_vgg(i["id"])
        total += 1

In [25]:
MEAN /= total
MEAN

array([0.02710865, 0.03218951, 0.05618101, ..., 0.03595013, 0.14719078,
       0.31317756], dtype=float32)

In [26]:
np.save(DIR + "vgg16/" + "MEAN.npy", MEAN)

In [27]:
MEAN.dtype

dtype('float32')